### Essential Libraries

In [1]:
import torch
import csv
import os
import time
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from dataloader import ATeX  # Ensure dataloader.py is in the same directory or in PYTHONPATH
import torchvision.models as models
from sklearn.metrics import classification_report  # Import the function
from torch.optim import SGD
import numpy as np
from torchvision.models import shufflenet_v2_x1_0
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import CosineAnnealingLR

### Dataset Preparation and DataLoader Setup

In [2]:
# Define transformations and dataset
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_std[0], mean_std[1]),
])

train_dataset = ATeX(split="train", transform=data_transforms)
test_dataset = ATeX(split="test", transform=data_transforms)

# Define DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Validation Dataset and DataLoader
val_dataset = ATeX(split="val", transform=data_transforms)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### Define the ShuffleNetV2 Model

In [3]:
# Define the number of classes
num_classes = len(train_dataset.classes)

# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize ShuffleNetV2 with pretrained weights
model = shufflenet_v2_x1_0(weights="DEFAULT")
model.fc = nn.Sequential(
    nn.Dropout(p=0.3),  # Increased dropout for regularization
    nn.Linear(model.fc.in_features, num_classes)
)

# Unfreeze earlier layers for fine-tuning
for name, param in model.named_parameters():
    if "fc" not in name:  # Skip the fully connected layers
        param.requires_grad = True

# Move the model to the appropriate device
model.to(device)


ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

### Training, Validation and Evaluation

In [4]:
# Initialize lists to store metrics
metrics = []

# Address to save the CSV file
model_name = "shufflenet_v2"
save_dir = r"D:/USC_Course/CSCE 790 Section 007 Neural Networks and Their Applications/atex-main/loss_training time/"
save_path = os.path.join(save_dir, f"training_metrics_{model_name}.csv")

# Ensure the directory exists
os.makedirs(save_dir, exist_ok=True)

# Define optimizer, scheduler, and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
criterion = nn.CrossEntropyLoss()  # Replace with label_smoothing_loss if needed

# Record the start time of the training process
training_start_time = time.time()

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    # Training Phase
    model.train()
    running_loss = 0.0
    for images, labels, _ in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)  # Use the defined loss function

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average training loss
    train_loss = running_loss / len(train_loader)

    # Update learning rate
    scheduler.step()

    # Validation Phase
    model.eval()
    val_loss = 0.0
    all_val_labels = []
    all_val_preds = []
    with torch.no_grad():
        for images, labels, _ in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            all_val_labels.extend(labels.cpu().numpy())
            all_val_preds.extend(preds.cpu().numpy())

    # Calculate validation metrics
    val_loss /= len(val_loader)
    val_report = classification_report(
        all_val_labels, all_val_preds, target_names=train_dataset.classes, zero_division=0, output_dict=True
    )
    val_precision = val_report['weighted avg']['precision']
    val_recall = val_report['weighted avg']['recall']
    val_f1 = val_report['weighted avg']['f1-score']

    # Save metrics for this epoch
    metrics.append({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "precision": val_precision,
        "recall": val_recall,
        "f1_score": val_f1
    })

    # Print training and validation metrics
    print(f"Epoch [{epoch+1}/{num_epochs}]: Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, "
          f"Recall: {val_recall:.4f}, F1-Score: {val_f1:.4f}")

# Record the end time of the training process
training_end_time = time.time()
total_training_time_seconds = training_end_time - training_start_time
total_training_time_minutes = total_training_time_seconds / 60

# Save metrics to a CSV file
with open(save_path, 'w', newline='') as csvfile:
    fieldnames = ["epoch", "train_loss", "val_loss", "precision", "recall", "f1_score"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(metrics)

# Append total training time in minutes
with open(save_path, 'a', newline='') as csvfile:
    csvfile.write(f"\nTotal Training Time (minutes):,{total_training_time_minutes:.2f}\n")

print(f"Training metrics saved to: {save_path}")
print(f"Total training time for all epochs: {total_training_time_minutes:.2f} minutes")


Epoch [1/30]: Train Loss: 2.5493, Val Loss: 2.3427, Precision: 0.1887, Recall: 0.2915, F1-Score: 0.1919
Epoch [2/30]: Train Loss: 2.1285, Val Loss: 1.9521, Precision: 0.3586, Recall: 0.4073, F1-Score: 0.3271
Epoch [3/30]: Train Loss: 1.7464, Val Loss: 1.4902, Precision: 0.5487, Recall: 0.5471, F1-Score: 0.4909
Epoch [4/30]: Train Loss: 1.3282, Val Loss: 1.0884, Precision: 0.6966, Recall: 0.6725, F1-Score: 0.6600
Epoch [5/30]: Train Loss: 0.9549, Val Loss: 0.8102, Precision: 0.7576, Recall: 0.7580, F1-Score: 0.7539
Epoch [6/30]: Train Loss: 0.7546, Val Loss: 0.7241, Precision: 0.7788, Recall: 0.7756, F1-Score: 0.7735
Epoch [7/30]: Train Loss: 0.6214, Val Loss: 0.6741, Precision: 0.7923, Recall: 0.7899, F1-Score: 0.7877
Epoch [8/30]: Train Loss: 0.5065, Val Loss: 0.5863, Precision: 0.8184, Recall: 0.8179, F1-Score: 0.8170
Epoch [9/30]: Train Loss: 0.4497, Val Loss: 0.5237, Precision: 0.8468, Recall: 0.8419, F1-Score: 0.8408
Epoch [10/30]: Train Loss: 0.3913, Val Loss: 0.5424, Precision: 

### Testing and Evaluation

In [5]:
# Testing the model
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels, _ in test_loader:  # Assuming your DataLoader provides 3 items
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Calculate classification metrics
class_names = train_dataset.classes  # Assuming train_dataset has the class names
report = classification_report(all_labels, all_preds, target_names=class_names)
print("Classification Report:\n")
print(report)

# Compute per-class accuracy
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)



Classification Report:

              precision    recall  f1-score   support

       delta       0.95      0.95      0.95       330
     estuary       0.86      0.89      0.87       125
       flood       0.87      0.87      0.87       283
    glaciers       0.96      0.94      0.95       240
  hot_spring       0.86      0.89      0.87       201
        lake       0.91      0.84      0.87       153
        pool       0.95      0.94      0.94        79
      puddle       0.79      0.78      0.78       168
      rapids       0.83      0.82      0.82       219
       river       0.76      0.77      0.76        73
         sea       0.81      0.89      0.85       108
        snow       0.90      0.87      0.88       142
       swamp       0.93      0.95      0.94       188
   waterfall       0.77      0.78      0.78        96
     wetland       0.98      0.95      0.96        93

    accuracy                           0.88      2498
   macro avg       0.87      0.87      0.87      2498
we

### Save Results

In [6]:
# Save classification report
with open("classification_report_shufflenetv2_model.txt", "w") as f:
    f.write(report)

# Save model weights
torch.save(model.state_dict(), "shufflenetv2_model_cpu.pth")